In [1]:
import scanpy as sc
import jax
from ot_pert.metrics import compute_metrics, compute_metrics_fast, compute_mean_metrics
import functools
import numpy as np

In [2]:
adata_preds_from_atac = sc.read("/lustre/groups/ml01/workspace/ot_perturbation/data/multiome/GLI3_KO_multiome_18d_adata_src_atac_with_predictions.h5ad")
adata_preds_from_rna = sc.read("/lustre/groups/ml01/workspace/ot_perturbation/data/multiome/GLI3_KO_multiome_18d_adata_src_with_predictions.h5ad")
adata_target_atac = sc.read_h5ad("/lustre/groups/ml01/workspace/ot_perturbation/data/multiome/GLI3_KO_multiome_18d_adata_tgt_test_processed.h5ad")
adata_atac_train = sc.read_h5ad("/lustre/groups/ml01/workspace/ot_perturbation/data/multiome/GLI3_KO_multiome_18d_adata_tgt_train_processed.h5ad")

/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [14]:
adata_preds_from_rna_unbalanced = sc.read("/lustre/groups/ml01/workspace/ot_perturbation/data/multiome/GLI3_KO_multiome_18d_adata_src_with_predictions_unbalanced.h5ad")


/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [3]:
adata_atac_train = sc.read_h5ad("/lustre/groups/ml01/workspace/ot_perturbation/data/multiome/GLI3_KO_multiome_18d_adata_tgt_train_processed.h5ad")

In [19]:
preds_from_atac = {}
preds_from_rna = {}
true_atac = {}
preds_from_rna_unbalanced = {}
for i in range(10):
    preds_from_atac[i] = adata_preds_from_atac.obsm[f"predicted_atac_{i}"][:,:30]
    true_atac[i] = adata_target_atac.obsm["X_pca"][:,:30]
    preds_from_rna[i] = adata_preds_from_rna.obsm[f"predicted_atac_{i}"][:, 10:40]
    preds_from_rna_unbalanced[i] = adata_preds_from_rna_unbalanced.obsm[f"predicted_atac_{i}"][:, 10:40]
    


In [28]:
preds_identity = {0: adata_preds_from_atac.obsm["X_pca"][:,:30]}

In [21]:
metrics_encoded_from_atac = jax.tree_util.tree_map(compute_metrics, preds_from_atac, true_atac)

In [22]:
compute_mean_metrics(metrics_encoded_from_atac)

{'r_squared': -0.5386099213755221,
 'sinkhorn_div_1': 26.055681133270262,
 'sinkhorn_div_10': 16.629411697387695,
 'sinkhorn_div_100': 13.295442962646485,
 'e_distance': 0.8559387105334437,
 'mmd': 0.011484599991222814}

In [23]:
metrics_encoded_from_rna = jax.tree_util.tree_map(compute_metrics, preds_from_rna, true_atac)

In [24]:
compute_mean_metrics(metrics_encoded_from_rna)

{'r_squared': -0.3697249694496364,
 'sinkhorn_div_1': 54.159423446655275,
 'sinkhorn_div_10': 44.8317626953125,
 'sinkhorn_div_100': 40.88667602539063,
 'e_distance': 0.4048078238511465,
 'mmd': 0.008571079101224354}

In [29]:
metrics_identity = jax.tree_util.tree_map(compute_metrics, preds_identity, {0: true_atac[0]})

In [30]:
compute_mean_metrics(metrics_identity)

{'r_squared': -0.05631441220782829,
 'sinkhorn_div_1': 7.723011493682861,
 'sinkhorn_div_10': 5.198238372802734,
 'sinkhorn_div_100': 2.7516136169433594,
 'e_distance': 2.9667363750196074,
 'mmd': 0.012982505842508712}

In [31]:
metrics_identity = jax.tree_util.tree_map(compute_metrics, preds_from_rna_unbalanced, true_atac)

In [32]:
compute_mean_metrics(metrics_identity)

{'r_squared': -0.3697249694496364,
 'sinkhorn_div_1': 54.159423446655275,
 'sinkhorn_div_10': 44.8317626953125,
 'sinkhorn_div_100': 40.88667602539063,
 'e_distance': 0.4048078238511465,
 'mmd': 0.008571079101224354}

In [15]:
metrics_encoded_from_rna

{0: {'r_squared': 0.186773556045167,
  'sinkhorn_div_1': 620266048.0,
  'sinkhorn_div_10': 359.1116638183594,
  'sinkhorn_div_100': 231.46322631835938,
  'e_distance': 1.7272871428227035,
  'mmd': 0.055085826717612606},
 1: {'r_squared': 0.18352470440932533,
  'sinkhorn_div_1': 2389987584.0,
  'sinkhorn_div_10': 356.0127868652344,
  'sinkhorn_div_100': 227.6110076904297,
  'e_distance': 1.7580845997567138,
  'mmd': 0.0554653851806148},
 2: {'r_squared': 0.16396652574037784,
  'sinkhorn_div_1': 632819810304.0,
  'sinkhorn_div_10': 401.33648681640625,
  'sinkhorn_div_100': 273.28106689453125,
  'e_distance': 1.8077618822780295,
  'mmd': 0.05549144560960642},
 3: {'r_squared': 0.20180889929033552,
  'sinkhorn_div_1': 828003200.0,
  'sinkhorn_div_10': 332.9805908203125,
  'sinkhorn_div_100': 203.94680786132812,
  'e_distance': 1.723044557429759,
  'mmd': 0.05581822539090261},
 4: {'r_squared': 0.16406000817552369,
  'sinkhorn_div_1': 13991589888.0,
  'sinkhorn_div_10': 378.1598815917969,
 